### Imports and model specifications

In [1]:
# Import dependencies
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F
import grid as gr

# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
alice.connect()
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
charlie = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")
bob.connect()
charlie.connect()
dan.connect()

gr.connect_all_nodes([bob, alice, charlie, dan])

In [2]:
# Support fetch plan + AST tensor
plan_func = False

if plan_func:
    @sy.func2plan(args_shape=[(1,)], state={"bias": th.tensor([3.0])})
    def plan_mult_3(x, state):
        bias = state.read("bias")
        x = x + bias
        return x
else:
    class Net(sy.Plan):
        def __init__(self):
            super(Net, self).__init__(id="net")
            self.fc1 = nn.Linear(1, 1)
            self.add_to_state(["fc1"])

        def forward(self, x):
            return self.fc1(x)
    
    plan_mult_3 = Net()
    plan_mult_3.build(th.tensor(1))

x = th.tensor([1.])
print(plan_mult_3(x))
    
sent_plan = plan_mult_3.send(alice).fix_prec().share(bob, charlie, crypto_provider=dan)

tensor([-0.1888], grad_fn=<AddBackward0>)


In [ ]:
from IPython.display import display_html

def restart_kernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
restart_kernel()

In [1]:
# Import dependencies
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F
import grid as gr

# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
alice.connect()
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
charlie = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")
bob.connect()
charlie.connect()
dan.connect()

gr.connect_all_nodes([bob, alice, charlie, dan])

In [2]:
# Fetch plan
fetched_plan = alice.fetch_plan("net")
x = th.tensor([1.])
x_ptr = x.fix_prec().share(bob, charlie, crypto_provider=dan)

# TODO: this should be stored automatically
me._objects[x_ptr.id] = x_ptr

In [3]:
# TODO: this should be done internally
id0, id1 = fetched_plan.state_ids

# TODO: we should not have direct access to the weights
a_sh = me._objects[id0].fix_prec().share(bob, charlie, crypto_provider=dan)
b_sh = me._objects[id1].fix_prec().share(bob, charlie, crypto_provider=dan)

# TODO: this should be stored automatically
me._objects[a_sh.id] = a_sh
me._objects[b_sh.id] = b_sh

In [4]:
new_state_ids = [a_sh.id, b_sh.id]

In [5]:
fetched_plan.replace_ids(fetched_plan.state_ids, new_state_ids)
fetched_plan.state_ids = new_state_ids

In [6]:
print(fetched_plan(x_ptr).get().float_prec())

tensor([-0.1890])


In [7]:
# Support fetching a plan
plan_func = False

if plan_func:
    @sy.func2plan(args_shape=[(1,)], state={"bias": th.tensor([3.0])})
    def plan_mult_3(x, state):
        bias = state.read("bias")
        x = x * bias
        return x
else:
    class Net(sy.Plan):
        def __init__(self):
            super(Net, self).__init__(id="net2")
            self.fc1 = nn.Linear(1, 1)
            self.add_to_state(["fc1"])

        def forward(self, x):
            return self.fc1(x)
    
    plan_mult_3 = Net()
    plan_mult_3.build(th.tensor(1))

sent_plan = plan_mult_3.send(alice)
print(sent_plan.id)

net2


In [ ]:
from IPython.display import display_html

def restart_kernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
restart_kernel()

In [1]:
# Import dependencies
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F
import grid as gr

# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
alice.connect()
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
charlie = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")
bob.connect()
charlie.connect()
dan.connect()

gr.connect_all_nodes([bob, alice, charlie, dan])

In [3]:
# Fetch plan
fetched_plan = alice.fetch_plan("net2")

x = th.tensor([1.])
print(fetched_plan(x))

tensor([0.8106], requires_grad=True)
